# Display mesh in 3D and using color on 2D

In [ ]:
import pandas as pd
import holoviews as hv

## Read in mesh from hgrid.gr3 file

In [ ]:
from schimpy import schism_mesh

smesh = schism_mesh.read_mesh('../tests/data/m1_hello_schism/hgrid.gr3')

In [ ]:
dfelems = pd.DataFrame(smesh.elems,columns=[0,1,2])
dfelems

replace 'z' with negative values for depth 

In [ ]:
dfnodes = pd.DataFrame(smesh.nodes, columns=['x','y','z'])
dfnodes['depth'] = -dfnodes.z
dfnodes = dfnodes.drop(columns=['z'])
dfnodes

## TriSurface drawn using nodes and elements

Adapted from https://anaconda.org/philippjfr/brain/notebook?version=2017.05.04.1924

The code below allows for the simplices already defined by elems to be used instead of doing a Delaunay triangulation (used from scipy as a way to calculate the simplices)

In [ ]:
# uncomment and the run install script below if plotly is not available
#!conda install -y -c conda-forge plotly

hv.extension('plotly')

import param

class TriSurface(hv.TriSurface):
    
    simplices = param.Array()

class TriSurfacePlot(hv.plotting.plotly.TriSurfacePlot):

    style_opts = ['cmap', 'plot_edges']

    def get_data(self, element, ranges, style, **kwargs):
        if element.simplices is None:
            return super(TriSurfacePlot, self).get_data(element, ranges, style, **kwargs)
        x, y, z = (element.dimension_values(i) for i in range(3))
        simplices = element.simplices
        return [dict(x=x, y=y, z=z, simplices=simplices)]
    
hv.Store.register({TriSurface: TriSurfacePlot}, 'plotly')

tris = TriSurface(dfnodes, simplices = dfelems.values).opts(width=800, height=800, cmap='blues_r')
tris.opts(plot_edges=False, colorbar=True)

Save mesh as html to embed in docs

In [ ]:
hv.save(tris,'mesh_surface_colored_by_depth.html')

## Trimesh to show z values with color
Alternate way of displaying the z (depth values) with color

In [ ]:
hv.extension('bokeh')
from holoviews.operation import datashader
trimesh = hv.TriMesh((dfelems.values, hv.Points(dfnodes,vdims='depth'))).opts(cmap='fire',node_alpha=0,edge_color='z',filled=True)
img = datashader.rasterize(trimesh).opts(cmap='rainbow4', colorbar=True, tools=['hover'], width=800)
img

In [ ]:
hv.save(img,'mesh_colored_by_depth.html')